## Day 7 Camel Cards

### Part A

**Global Variables**

- ```hands``` is a ranking (from highest to lowest) of all the possible card hands.
- ```cards``` is a ranking (from highest to lowest) of all the possible cards.
- ```ranked_cards``` is a list of all of our inputted hands, ranked from highest to lowest.
- ```hand_to_bid``` maps each hand to its corresponding bid.

**Functions**

1. ```pattern_match(s: str)``` takes a hand of cards `s` in the form of a string and returns what type of hand it is (e.g., five of a kind, four of a kind, full house etc.). It does this by storing a Counter object and a set of all the characters in s, and using the properties of each hand to classify `s` by its hand type.
2. ```compare_hands(a: str, b: str)``` compares the strengths of hand `a` and hand `b`, which are both strings.
3. ```rank_insert(arr: list, s: str)``` inserts a hand `s` into an array `arr` (which is ordered from highest to lowest hand strength), such that the order of the array is maintained.

**Main Program**

1. Iterate through all of the lines in the problem input file and insert them into `ranked_cards` using our defined functions, whilst storing their corresponding bids in `hand_to_bid` which takes the hand as a key, and maps it to the bid as a value. 
2. We then iterate through `ranked_cards` and calculate the final answer.

In [71]:
# Part A

from collections import Counter

file = open('day_7_part_1.txt', 'r')

lines = list(map(lambda x: x.strip(), file.readlines()))   

hands = ['Five of a kind', 'Four of a kind', 'Full house', 'Three of a kind', 'Two pair', 'One pair', 'High card']
cards = 'AKQJT98765432'
ranked_cards = []
hand_to_bid = {}

def pattern_match(s: str):

    count_s = Counter(s)
    set_s = set(s)

    if len(set_s) == 1:
        return 'Five of a kind'
    elif len(set_s) == 5:
        return 'High card'
    elif 4 in count_s.values():
        return 'Four of a kind'
    else:
        if 3 in count_s.values():
            if len(set_s) == 2:
                return 'Full house'
            else:
                return 'Three of a kind'
        else:
            if len(set_s) == 4:
                return 'One pair'
            else:
                return 'Two pair'

def compare_hands(a: str, b: str):

    ai = hands.index(pattern_match(a))
    bi = hands.index(pattern_match(b))

    if ai < bi:
        return a
    elif ai == bi:
        for i in range(5):
            if cards.index(a[i]) < cards.index(b[i]):
                return a
            elif cards.index(a[i]) > cards.index(b[i]):
                return b
            else:
                continue

    else:
        return b

def rank_insert(arr: list, s: str):
    
    if not arr:
        return [s]

    if compare_hands(arr[0], s) == arr[0]:
        arr = [arr[0]] + rank_insert(arr[1:], s)
        return arr
    else:
        return [s] + arr

while lines:
    hand, bid = lines.pop(0).split(' ')
    bid = int(bid)
    hand_to_bid[hand] = bid
    ranked_cards = rank_insert(ranked_cards, hand)

ans = 0
i = len(ranked_cards)
for j, h in enumerate(ranked_cards):
    ans += (i-j)*hand_to_bid[h]

ans

247823654

### Part B

**Changes from Part A**

- `cards` has 'J' at the end instead of where a 'Jack' would normally be, signifying that as a 'Joker' card, it has the lowest priority.
- `pattern_match(s: str)` now handles the instances of a 'Joker' card:
    - If the frequency of the 'Joker' card in the hand is a minority, we can greedily assign it to the highest priority card (e.g., 'A' would have higher priority than '8') which occurs the most often, so as to maximise the strength of the hand.
    - If the frequency of the 'Joker' card in the hand is a majority, we find the card in the hand with the next-highest frequency and assign all the 'Jokers' to that card.
    - If the entire hand is 5 'Jokers', then we make it all aces, since there are no non-'Joker' cards in the hand to assign the 'Jokers' to.
    - Using the same methods as Part A's `pattern_match(s: str)` we can classify the type of hand.
    - Because we only return the type of hand instead of the modified `wild_s`, all other functions work as normal.

In [70]:
# Part B

from collections import Counter

file = open('day_7_part_1.txt', 'r')

lines = list(map(lambda x: x.strip(), file.readlines()))   

hands = ['Five of a kind', 'Four of a kind', 'Full house', 'Three of a kind', 'Two pair', 'One pair', 'High card']
cards = 'AKQT98765432J'
ranked_cards = []
hand_to_bid = {}

def pattern_match(s: str):

    wild_s = ''
    count_s = Counter(s)
    _max = (0, 0) #card, number of occurrences. Find the highest frequency and magnitude card and greedily assign the Jokers to it.
    for k, occ in count_s.items():
        if occ == _max[1]:
            if cards.index(_max[0]) < cards.index(k):
                _max = (_max[0], occ)
            else:
                _max = (k, occ)
            
        if occ > _max[1]:
            _max = (k, occ)

    if _max[0] == 'J':
        # _max = ('A', _max[1])
        items = sorted(count_s.items(), key=lambda x:x[1], reverse=True)
        for k, v in items:
            if k == 'J': 
                continue
            else:
                _max = (k, _max[1])
                break

    if 'J' in s:
        wild_s = [char if char != 'J' else _max[0] for char in s]

        if s == 'JJJJJ':
            wild_s = 'AAAAA'

        count_s = Counter(wild_s)
        set_s = set(wild_s)

    else:
        count_s = Counter(s)
        set_s = set(s)
    
    if len(set_s) == 1:
        return 'Five of a kind'
    elif len(set_s) == 5:
        return 'High card'
    elif 4 in count_s.values():
        return 'Four of a kind'
    else:
        if 3 in count_s.values():
            if len(set_s) == 2:
                return 'Full house'
            else:
                return 'Three of a kind'
        else:
            if len(set_s) == 4:
                return 'One pair'
            else:
                return 'Two pair'

def compare_hands(a: str, b: str):

    ai = hands.index(pattern_match(a))
    bi = hands.index(pattern_match(b))

    if ai < bi:
        return a
    elif ai == bi:
        for i in range(5):
            if cards.index(a[i]) < cards.index(b[i]):
                return a
            elif cards.index(a[i]) > cards.index(b[i]):
                return b
            else:
                continue

    else:
        return b

def rank_insert(arr: list, s: str):
    
    if not arr:
        return [s]

    if compare_hands(arr[0], s) == arr[0]:
        arr = [arr[0]] + rank_insert(arr[1:], s)
        return arr
    else:
        return [s] + arr

while lines:
    hand, bid = lines.pop(0).split(' ')
    bid = int(bid)
    hand_to_bid[hand] = bid
    ranked_cards = rank_insert(ranked_cards, hand)

ans = 0
i = len(ranked_cards)
for j, h in enumerate(ranked_cards):
    ans += (i-j)*hand_to_bid[h]

ans #245461700

245461700